# APIM ❤️ OpenAI

## OpenAI Tracing tests

Invoke OpenAI API with trace enabled and returns the Azure API Management [tracing information](https://learn.microsoft.com/azure/api-management/api-management-howto-api-inspector). Tracing helps you debug and troubleshoot your API.

### Prerequisites

- [Python 3.12 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Python environment](https://code.visualstudio.com/docs/python/environments#_creating-environments) with the [requirements.txt](../../requirements.txt) or run `pip install -r requirements.txt` in your terminal
- [An Azure Subscription](https://azure.microsoft.com/free/) with [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#contributor) + [RBAC Administrator](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#role-based-access-control-administrator) or [Owner](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#owner) roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and [Signed into your Azure subscription](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

▶️ Click `Run All` to execute all steps sequentially, or execute them `Step by Step`...


<a id='0'></a>
### ⚙️ Initialize client tool for a given APIM service

👉 An existing Azure OpenAI API is expected to be already configured on APIM

In [ ]:
import os, sys, json, requests
sys.path.insert(1, '../shared')  # add the shared directory to the Python path
import utils
from apimtools import APIMClientTool

openai_deployment_name = "gpt-4o-mini"
openai_api_version = "2024-10-21"

apimClientTool = APIMClientTool(
    "change-me", ## specify the existing API Management resource name
    "change-me" ## specify the resource group name where the API Management resource is located
)
apimClientTool.initialize()
apimClientTool.discover_openai_api('openai')



<a id='requests'></a>
### 🧪 Test the API using a direct HTTP call


In [ ]:
apim_debug_authorization = apimClientTool.get_debug_credentials("PT1H")
url = f"{apimClientTool.apim_resource_gateway_url}/{apimClientTool.openai_api_path}/deployments/{openai_deployment_name}/chat/completions?api-version={openai_api_version}"

messages={"messages":[
    {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]}
response = requests.post(url, headers = {'api-key':apimClientTool.apim_subscriptions[1].get("key"), 'Apim-Debug-Authorization': apim_debug_authorization}, json = messages)
print("status code: ", response.status_code)
trace_id = response.headers.get("Apim-Trace-Id")
print("Apim-Trace-Id: ", trace_id) # this header will be used to get API trace details
print("headers ", response.headers)
print("x-ms-region: ", response.headers.get("x-ms-region")) # this header is useful to determine the region of the backend that served the request
if (response.status_code == 200):
    data = json.loads(response.text)
    print("💬 ", data.get("choices")[0].get("message").get("content"))
else:
    print(response.text)


<a id='trace1'></a>
### 🔍 Analyze the API trace from direct HTTP call

With the following request we will get the json with the complete trace information.

👉 Customize the Notebook layout with scrolling to access the full output trace

In [ ]:
print(json.dumps(apimClientTool.get_trace(trace_id), indent=4)) # this will get the trace details for the request made above

<a id='sdk'></a>
### 🧪 Test with the Azure OpenAI Python SDK


In [ ]:
import time
from openai import AzureOpenAI
messages=[
        {'role': 'user', 'content': 'Count to 100, with a comma between each number and no newlines. E.g., 1, 2, 3, ...'}
]
apim_debug_authorization = apimClientTool.get_debug_credentials("PT1H")

messages=[
    {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]
client = AzureOpenAI(
    azure_endpoint=apimClientTool.apim_resource_gateway_url,
    api_key=apimClientTool.apim_subscriptions[1].get("key"),
    api_version=openai_api_version
)
response = client.chat.completions.with_raw_response.create(model=openai_deployment_name, messages=messages, extra_headers={'Apim-Debug-Authorization': apim_debug_authorization})
trace_id = response.headers.get("Apim-Trace-Id")
print("Apim-Trace-Id: ", trace_id) # this header will be used to get API trace details
print("headers ", response.headers)
print("x-ms-region: ", response.headers.get("x-ms-region")) # this header is useful to determine the region of the backend that served the request

completion = response.parse() 
print("💬 ",completion.choices[0].message.content)


<a id='trace2'></a>
### 🔍 Analyze the API trace from the SDK call

In [ ]:
print(json.dumps(apimClientTool.get_trace(trace_id), indent=4)) # this will get the trace details for the request made above